# Training linear models
In this notebook, we will first start by looking at the Linear Regression model. The model will be trained in 2 ways:
1. Using a direct 'closed-form' equation - that directly computes the model parameters best fit to the training set.
2. Using an iterative optimization approach called Gradient Descent(GD) - that gradually tweaks the model parameters to minimize the cost function over the training set, eventually converging on the same set of parameters as the first method.

Then we will look at Polynomial Regression, a model that can fit non-linear datasets.
<br><br>
Then we will look at several Regularization techniques that can reduce the risk of overfitting the training sets.
<br><br>
Finally, we will look at 2 more models : Logistic Regression and Softmax Regression.

## Setup

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images")
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

## Linear Regression

### Using the normal equation